In [ ]:
#!pip install polars -q

In [1]:
import polars as pl
from warnings import filterwarnings

filterwarnings('ignore')

# <strong style="color:#5e17eb">I. Data Frame  </strong>


<strong style="color:#5e17eb"> Parquet </strong>

In [ ]:
# df.write_parquet(r"C:\Users\Rudra\Desktop\yelp\parquet-data\business.parquet")

In [4]:
df = pl.DataFrame()
df = pl.read_parquet(r"C:\Users\Rudra\Desktop\yelp\parquet-data\business.parquet")

In [3]:
df.sample()

business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
str,str,str,str,str,str,f64,f64,f64,i64,i64,str,str,str
"""nyq0Z7rAx3kiD-HxEY4Oeg""","""Salty Dog""","""8502 Manchester Rd""","""Saint Louis""","""MO""","""63144""",38.612232,-90.342895,3.5,5,0,null,"""Pets, Pet Services, Pet Sittin…",null


In [ ]:
dff = df.clone()

#  <strong style="color:#5e17eb">  II. DataFrame Attributes & Methods</strong>


## <strong style="color:#5e17eb"> is_duplicated() </strong>


In [40]:
# df[df.is_duplicated()] this is the pandas style
df.filter(df.is_duplicated())

business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
str,str,str,str,str,str,f64,f64,f64,i64,i64,str,str,str


## <strong style="color:#5e17eb"> is_empty() </strong>


In [42]:
df.is_empty()

False

## <strong style="color:#5e17eb"> extend() </strong>
- When the we merge to data quickly

In [ ]:
# df.extend(other_df)

## <strong style="color:#5e17eb">  explode() </strong>


- What: Turns list-like column into multiple rows.

- Modifies Original?  No (returns new df).

- Why: To flatten nested lists.

In [55]:
pl.DataFrame({"a": [[1, 2], [3]]}).explode("a")
# Output: 1 row → 2 rows

a
i64
1
2
3


## <strong style="color:#5e17eb"> cast() </strong>


In [63]:
df.schema

Schema([('business_id', String),
        ('name', String),
        ('address', String),
        ('city', String),
        ('state', String),
        ('postal_code', String),
        ('latitude', Float64),
        ('longitude', Float64),
        ('stars', Float64),
        ('review_count', Int64),
        ('is_open', Int64),
        ('attributes', String),
        ('categories', String),
        ('hours', String)])

In [67]:
df.with_columns(
    pl.col("stars").cast(pl.Float32)
).schema

Schema([('business_id', String),
        ('name', String),
        ('address', String),
        ('city', String),
        ('state', String),
        ('postal_code', String),
        ('latitude', Float64),
        ('longitude', Float64),
        ('stars', Float32),
        ('review_count', Int64),
        ('is_open', Int64),
        ('attributes', String),
        ('categories', String),
        ('hours', String)])

## <strong style="color:#5e17eb">  approx_n_unique()</strong>


In [ ]:
df.select(pl.all().approx_n_unique()).collect()

In [68]:
df.approx_n_unique()


business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
149577,114011,120883,1395,27,3330,135521,131885,9,1160,2,87437,82225,49324


## <strong style="color:#5e17eb">  fill_nan() & fill_null()</strong>

| Function      | Fills | Type Targeted | Example Use                    |
| ------------- | ----- | ------------- | ------------------------------ |
| `fill_nan()`  | NaN   | Only float    | Replace invalid floats         |
| `fill_null()` | null  | Any dtype     | Replace `None`, missing values |


| Property | `null`               | `NaN`                    |
| -------- | -------------------- | ------------------------ |
| Type     | Missing value        | Invalid float (IEEE 754) |
| Dtypes   | Any (int, str, etc.) | Only floats              |
| Meaning  | “No value at all”    | “Not a number”           |


## <strong style="color:#5e17eb"> height() & weight() </strong>
- Number of rows

In [69]:
print(df.height)  # Same as df.shape[0]


150346


In [ ]:
print(df.width)

## <strong style="color:#5e17eb"> interpolate()  </strong>

- What: Fill missing numeric values linearly.

- Why: For time-series or ordered data gaps.

## <strong style="color:#5e17eb"> hash_rows() </strong>


- What: Create hash values for each row (like fingerprints).

- Why: Fast comparisons, joins, deduplication.

In [71]:
df.hash_rows()

""
u64
1016177515689230841
9535693652620671175
13099489936039061843
3073835934067927028
12733077377254317125
…
2387885169568056652
11105544557764227190
17158366205228388882


## <strong style="color:#5e17eb"> flags  </strong>


In [76]:
g = pl.DataFrame({
    "col1": [1, 2, 3],
    "col2": ["a", "b", "c"]
})

g.flags


{'col1': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'col2': {'SORTED_ASC': False, 'SORTED_DESC': False}}

In [73]:
df.flags

{'business_id': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'name': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'address': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'city': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'state': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'postal_code': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'latitude': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'longitude': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'stars': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'review_count': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'is_open': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'attributes': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'categories': {'SORTED_ASC': False, 'SORTED_DESC': False},
 'hours': {'SORTED_ASC': False, 'SORTED_DESC': False}}

## <strong style="color:#5e17eb"> item() </strong>
- Only for series
- What: Return Python scalar from 1-element Series.

In [ ]:
pl.Series([42]).item()

42

## <strong style="color:#5e17eb"> lazy()  </strong>

- What: Convert eager DataFrame to LazyFrame.

- Why: For optimization and deferred execution.

In [80]:
lf = df.lazy()

## <strong style="color:#5e17eb"> limit(n)  </strong>


In [81]:
df.limit(3)

business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
str,str,str,str,str,str,f64,f64,f64,i64,i64,str,str,str
"""Pns2l4eNsfO8kk83dixA6A""","""Abby Rappoport, LAC, CMQ""","""1616 Chapala St, Ste 2""","""Santa Barbara""","""CA""","""93101""",34.426679,-119.711197,5.0,7,0,"""{'ByAppointmentOnly': 'True'}""","""Doctors, Traditional Chinese M…",null
"""mpf3x-BjTdTEA3yCZrAYPw""","""The UPS Store""","""87 Grasso Plaza Shopping Cente…","""Affton""","""MO""","""63123""",38.551126,-90.335695,3.0,15,1,"""{'BusinessAcceptsCreditCards':…","""Shipping Centers, Local Servic…","""{'Monday': '0:0-0:0', 'Tuesday…"
"""tUFrWirKiKi_TAnsVWINQQ""","""Target""","""5255 E Broadway Blvd""","""Tucson""","""AZ""","""85711""",32.223236,-110.880452,3.5,22,0,"""{'BikeParking': 'True', 'Busin…","""Department Stores, Shopping, F…","""{'Monday': '8:0-22:0', 'Tuesda…"


## <strong style="color:#5e17eb"> max() </strong>

In [82]:
max_df_ex = pl.DataFrame({
    "A": [1, 2, 3],
    "B": [6, 7, 8],
    "C": ["apple", "banana", "cherry"],
})

max_values = max_df_ex.max()
print(max_values)


shape: (1, 3)
┌─────┬─────┬────────┐
│ A   ┆ B   ┆ C      │
│ --- ┆ --- ┆ ---    │
│ i64 ┆ i64 ┆ str    │
╞═════╪═════╪════════╡
│ 3   ┆ 8   ┆ cherry │
└─────┴─────┴────────┘


-  max() function on a string column (or a Series of strings), it determines the "maximum" value based on *lexicographical* (dictionary) order. 

In [83]:
df.max()

business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
str,str,str,str,str,str,f64,f64,f64,i64,i64,str,str,str
"""zzyx5x0Z7xXWWvWnZFuxlQ""","""​​Transformational Abdominal M…","""​185 E State St""","""​Lithia""","""XMS""","""T9E 0V3""",53.679197,-73.200457,5.0,7568,1,"""{'WiFi': 'None'}""","""Zoos, Tours, Arts & Entertainm…","""{'Wednesday': '9:30-20:0', 'Th…"


## <strong style="color:#5e17eb"> max_horizontal()  </strong>
- Row-wise maximum

In [86]:
max_hor_df = pl.DataFrame({
    "col1": [1, 5, 2],
    "col2": [8, 3, 6],
})

row_wise_max = max_hor_df.max_horizontal()
print(row_wise_max)


shape: (3,)
Series: 'max' [i64]
[
	8
	5
	6
]


## <strong style="color:#5e17eb"> map_rows() </strong>
- What: Row-wise Python function application.

- Warning: Slower than expressions. Use only when necessary.

In [ ]:
# df.map_rows(lambda row: row[0] + row[1])

## <strong style="color:#5e17eb"> n_unique()  </strong>


In [ ]:
# number of unique values
df["stars"].n_unique()

9

## <strong style="color:#5e17eb"> n_chunks </strong>
- What: Shows number of memory chunks.

- Why: Internal info on memory layout.

- Use: For debugging performance.

In [7]:
df.n_chunks()

12

In [ ]:
print(f"Number of chunks in DataFrame (first series): {df.n_chunks(strategy='first')}")
print(f"Number of chunks in DataFrame (all series): {df.n_chunks(strategy='all')}")

Number of chunks in DataFrame (first series): 12
Number of chunks in DataFrame (all series): [12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]


<strong style="color:#5e17eb"> Summary </strong>

- Now, imagine that building the whole castle at once is hard, especially if it's super big, like a giant LEGO city! It might even be too much for your hands to hold all the pieces at once.
- It means that every column in your Polars DataFrame is currently stored in 12 separate chunks or memory allocations
- In summary, the output indicates that your DataFrame's columns are uniformly chunked. This is generally a reasonable state. However, it's a good practice to be aware of the concept of chunking and the availability of `.rechunk()` for potential performance optimization if needed. 

## <strong style="color:#5e17eb"> Pipe </strong>


- It lets you pass a DataFrame through a function as part of a chain.

- Makes your code modular and readable

- Clean chaining of custom Python logic with Polars expressions

In [ ]:
def double_stars(df): 
    return df.with_columns((pl.col("stars") * 2).alias("double"))

df.pipe(double_stars)

business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,double
str,str,str,str,str,str,f64,f64,f64,i64,i64,str,str,str,f64
"""Pns2l4eNsfO8kk83dixA6A""","""Abby Rappoport, LAC, CMQ""","""1616 Chapala St, Ste 2""","""Santa Barbara""","""CA""","""93101""",34.426679,-119.711197,5.0,7,0,"""{'ByAppointmentOnly': 'True'}""","""Doctors, Traditional Chinese M…",null,10.0
"""mpf3x-BjTdTEA3yCZrAYPw""","""The UPS Store""","""87 Grasso Plaza Shopping Cente…","""Affton""","""MO""","""63123""",38.551126,-90.335695,3.0,15,1,"""{'BusinessAcceptsCreditCards':…","""Shipping Centers, Local Servic…","""{'Monday': '0:0-0:0', 'Tuesday…",6.0
"""tUFrWirKiKi_TAnsVWINQQ""","""Target""","""5255 E Broadway Blvd""","""Tucson""","""AZ""","""85711""",32.223236,-110.880452,3.5,22,0,"""{'BikeParking': 'True', 'Busin…","""Department Stores, Shopping, F…","""{'Monday': '8:0-22:0', 'Tuesda…",7.0
"""MTSW4McQd7CbVtyjqoe9mw""","""St Honore Pastries""","""935 Race St""","""Philadelphia""","""PA""","""19107""",39.955505,-75.155564,4.0,80,1,"""{'RestaurantsDelivery': 'False…","""Restaurants, Food, Bubble Tea,…","""{'Monday': '7:0-20:0', 'Tuesda…",8.0
"""mWMc6_wTdE0EUBKIGXDVfA""","""Perkiomen Valley Brewery""","""101 Walnut St""","""Green Lane""","""PA""","""18054""",40.338183,-75.471659,4.5,13,1,"""{'BusinessAcceptsCreditCards':…","""Brewpubs, Breweries, Food""","""{'Wednesday': '14:0-22:0', 'Th…",9.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""IUQopTMmYQG-qRtBk-8QnA""","""Binh's Nails""","""3388 Gateway Blvd""","""Edmonton""","""AB""","""T6J 5H2""",53.468419,-113.492054,3.0,13,1,"""{'ByAppointmentOnly': 'False',…","""Nail Salons, Beauty & Spas""","""{'Monday': '10:0-19:30', 'Tues…",6.0
"""c8GjPIOTGVmIemT7j5_SyQ""","""Wild Birds Unlimited""","""2813 Bransford Ave""","""Nashville""","""TN""","""37204""",36.115118,-86.766925,4.0,5,1,"""{'BusinessAcceptsCreditCards':…","""Pets, Nurseries & Gardening, P…","""{'Monday': '9:30-17:30', 'Tues…",8.0
"""_QAMST-NrQobXduilWEqSw""","""Claire's Boutique""","""6020 E 82nd St, Ste 46""","""Indianapolis""","""IN""","""46250""",39.908707,-86.065088,3.5,8,1,"""{'RestaurantsPriceRange2': '1'…","""Shopping, Jewelry, Piercing, T…",null,7.0


In [ ]:
def extract_categories(df):
    return df.with_columns(
        pl.col("categories")
        .str.split(", ")
        .alias("categories_list")
    )

df = df.pipe(extract_categories)

In [15]:
df["categories_list"]

categories_list
list[str]
"[""Doctors"", ""Traditional Chinese Medicine"", … ""Nutritionists""]"
"[""Shipping Centers"", ""Local Services"", … ""Printing Services""]"
"[""Department Stores"", ""Shopping"", … ""Furniture Stores""]"
"[""Restaurants"", ""Food"", … ""Bakeries""]"
"[""Brewpubs"", ""Breweries"", ""Food""]"
…
"[""Nail Salons"", ""Beauty & Spas""]"
"[""Pets"", ""Nurseries & Gardening"", … ""Shopping""]"
"[""Shopping"", ""Jewelry"", … ""Fashion""]"


## <strong style="color:#5e17eb">  Product</strong>


In [19]:
df.select(pl.col("stars").product())


stars
f64
inf


# <strong style="color:#5e17eb"> III. Summary Table </strong>

| Method              | What it Does                           | 
| ------------------- | -------------------------------------- | 
| `extend()`          | Append another df (in-place)           |
| `explode()`         | Flatten list column to rows            | 
| `cast()`            | Change dtype                           |
| `approx_n_unique()` | Fast estimate of unique rows           | 
| `fill_nan()`        | Fill only float NaNs                   | 
| `fill_null()`       | Fill nulls (any type)                  | 
| `fold()`            | Row-wise reduce columns                | 
| `height`            | Row count                              |
| `interpolate()`     | Fill missing values linearly           | 
| `hash_rows()`       | Hash each row                          | 
| `item()`            | Get single value from 1-element Series | 
| `lazy()`            | Convert eager to lazy                  | 
| `limit()`           | First `n` rows                         | 
| `max()`             | Max values per column                  | 
| `map_rows()`        | Row-wise function (slow!)              | 
| `n_unique()`        | Unique value count                     | 
| `n_chunks()`        | Memory layout info                     | 
| `pipe()`            | Chainable functions                    | 
| `product()`         | Multiply all values in a column        | 


<div style="text-align: center;">
  <h4 style="
    display: inline-block;
    color: #5e17eb;
    font-family: 'Segoe UI';
    border-left: 5px solid #729be8ff;
    background-color: #F8F9F9;
    padding: 10px 20px;
    border-radius: 5px;
    text-align: left;
  "><b>
    Thank You 💜
    </b>
  </h4>
</div>